<a href="https://colab.research.google.com/github/SilvioCesarTeixeira/EstudoCorrLinearPython/blob/main/Estat%C3%ADsticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
##from google.colab import files
##uploaded = files.upload()
import statsmodels.api as sm
import pandas.util.testing as tm
import io

Entrada de Dados

In [ ]:
# entrada com listas separadas
# x = []
# y = []
#
# entrada com arquivo (exemplo csv)
# df = pd.read.csv(io.StringIO(uploaded['nome_do_arquivo.csv'].decode('utf-8)))
#
# entrada com listas simples, criando Dataframe, renomeando colunas para x e y
# lstx = ()
# lsty = ()
# df = pd.DataFrame(list(zip(lstx,lsty)), columns = "x","y")
#
# df.head(10)

Criar dataframe

In [ ]:
df1 = pd.DataFrame(data=df)
#definir a variável resposta
y = df1['nome_da_coluna_y']
#definir a variável preditora
x = df1['nome_da_coluna_x']

Realizar análise de regressão linear com API statsmodel

In [ ]:
#adicionar uma constante preditora
x = sm.add_constant(x)
#utilizar linear regression model
modelo = sm.OLS(y, x).fit() ## atenção esta variável será usada no IC
print(modelo.summary())

Visualizar comportamento graficamente

In [ ]:
#@title  { display-mode: "code" }
sns.lmplot(x='nome_da_coluna_x', y='nome_da_coluna_y', data=df1);plt.grid(True)

Verificar Beta zero e Beta um

In [ ]:
# parâmetros da reta
b=modelo.params
b0=b[0] #intercepto
b1=b[1] #inclinação
print('y= {0}{1} x'.format(b0,b1))

Calcular erros e erros padronizados

In [ ]:
x = df1['nome_da_coluna_x']

In [ ]:
y_pred=list(modelo.predict())
#sns.resid(modelo)
resi=list(modelo.resid)

In [ ]:
#criar instancia influence
influence = modelo.get_influence()
#obter residuos standardizados
stdresid = list(influence.resid_studentized_internal)

In [ ]:
prop=np.divide(resi,stdresid) #e/e* para verificar a proporcionalidade entre o erro e o erro padronizado

Criar dataframe com os erros padronizados e proporcionais

In [ ]:
# title Tabela geral
dft = pd.DataFrame(list(zip(x, y, y_pred, resi, stdresid, prop)),
               columns =["x","y","yc","e","e*","e/e*"])
dft 

In [ ]:
#@title Gráficos de diagnóstico 2 (y vs. yc)
sns.lmplot(x='y', y='yc', data=dft);plt.grid(True)

In [ ]:
#@title Gráficos de diagnóstico 3 (resíduos padronizados vs. x)
sns.scatterplot(x='x', y='e*', data=dft);plt.grid(True)
plt.xlabel('x= Rótulo da variável preditora')
plt.ylabel('e*= Resíduos padronizados')
plt.axhline(y=0, color='black', linestyle='--', linewidth=1)
plt.show()

In [ ]:
#@title Gráficos de diagnóstico 4 (resíduos padronizados vs. y)
sns.scatterplot(x='y', y='e*', data=dft);plt.grid(True)
plt.xlabel('y= Rótulo da variável dependente')
plt.ylabel('e*= Resíduos padronizados')
plt.axhline(y=0, color='black', linestyle='--', linewidth=1)
plt.show()

In [ ]:
#@title Gráficos de diagnóstico 5 Probabilidade Normal
QQ = ProbPlot(influence.resid_studentized_internal)
plot_lm_2 = QQ.qqplot(line='45', alpha=0.5, color='#4C72B0', lw=1)
plot_lm_2.axes[0].set_xlabel('Percentil')
plot_lm_2.axes[0].set_ylabel('Resíduos padronizados')

In [ ]:
#@title Teste de Outliers
test = modelo.outlier_test()
print("test.columns:", test.columns)
print(test)
outliers = ((x[i],y[i]) for i,t in enumerate(test.iloc[:,2]) if t < 0.5)
print ('Outliers: ', list(outliers))

Visualizar graficamente dados identificados

In [ ]:
plt.rcParams.update({'figure.figsize':(10,8), 'figure.dpi':100})
sns.lmplot(x='nome_da_coluna_x', y='nome_da_coluna_y',hue='nome_da_coluna_identificadora', data=df1);

Calcular Intervalo de Confiança por meio do valor retornado da API statsmodel

In [ ]:
a= modelo.conf_int(alpha=0.05, cols=None) #intervalo de confiança de b0 e b1
a

In [ ]:
display(a.iloc[0,:]) #intervalo de confiança beta0
display(a.iloc[1,:]) #intervalo de confiança beta1